In [13]:
import pandas as pd
import math
import os
from pathlib import Path

KAISAI_DATE = "20260215"
year = KAISAI_DATE[:4]

RESULT_DIR = Path(".", "prof_result", year)
INPUT_CSV = f"results_prof_index_{KAISAI_DATE}.csv"
OUT_DIR = "target_marks_out"

TARGET_DIR = Path(OUT_DIR , year, KAISAI_DATE)

marks_top5 = ["◎", "○", "▲", "△", "★"]

In [14]:
def split_int_to_17_chunks(n: int):
    """nを最大3つの（<=17）に分割。残りが出る（>51）場合は切り捨て運用にする。"""
    chunks = []
    rem = max(0, int(n))
    for _ in range(3):
        c = min(17, rem)
        chunks.append(c)
        rem -= c
    return chunks  # [m5, m6, m7]

In [15]:
os.makedirs(TARGET_DIR, exist_ok=True)

df = pd.read_csv(RESULT_DIR / INPUT_CSV, encoding="cp932")
df["総合利益度"] = pd.to_numeric(df["総合利益度"], errors="coerce")

df["M4"] = ""
df["M5"] = ""
df["M6"] = ""
df["M7"] = ""

# レース単位（場所+R）で、総合利益度>0だけを降順ランキング
for (_, _), g in df.groupby(["場所","R"], sort=False):
    pos = g[g["総合利益度"].notna() & (g["総合利益度"] > 0)].copy()
    if pos.empty:
        continue
    pos = pos.sort_values("総合利益度", ascending=False)

    for rank, (idx, row) in enumerate(pos.iterrows(), start=1):
        # 馬印4：上位1～5は◎○▲△★、6位以降は×
        df.loc[idx, "M4"] = marks_top5[rank-1] if rank <= 5 else "×"

        # 馬印5～7：整数部を最大17で分割（例：38 → 17,17,04）
        n = int(math.floor(row["総合利益度"]))
        m5, m6, m7 = split_int_to_17_chunks(n)

        if m5 >= 0: df.loc[idx, "M5"] = f"{m5:02d}"
        if m6 > 0: df.loc[idx, "M6"] = f"{m6:02d}"
        if m7 > 0: df.loc[idx, "M7"] = f"{m7:02d}"

# TARGET取り込み用（馬名,ワークデータ）※ヘッダなし
def export(mark_col, filename):
    out = df[["馬名", mark_col]].rename(columns={mark_col: "ワークデータ"})
    out.to_csv(os.path.join(TARGET_DIR, filename), index=False, header=False, encoding="cp932")

export("M4", f"work_for_mark4_{KAISAI_DATE}.csv")
export("M5", f"work_for_mark5_{KAISAI_DATE}.csv")
export("M6", f"work_for_mark6_{KAISAI_DATE}.csv")
export("M7", f"work_for_mark7_{KAISAI_DATE}.csv")

# 検算用（任意）
df[["場所","R","馬番","馬名","総合利益度","M4","M5","M6","M7"]].to_csv(
    os.path.join(OUT_DIR, "computed_marks_check.csv"),
    index=False, encoding="utf-8"
)